#Compositionality examples with sequence neural networks

Can neural network models handle recursive compositional interpretation? One can check this compositional inferences. For example, if _Garry's parent_ refers to Bill, and _Bill's friend_ refers to Donna, _Garry's parent's friend_ must refer to Donna as well.

First, we can download code from the paper _On Learning Interpreted Languages with Recurrent Models_ (https://aclanthology.org/2022.cl-2.7/):

In [ ]:
!git clone https://github.com/dpaperno/LSTM_composition

We can now run a script with different parameters. The script randomly assigns relations between individuals (Ann, Bill, Donna, and Garry), and trains a small neural network model to choose the correct referent to a phrase like _Garry's parent_. The neural network is trained on phrases with at most 2 common nouns (e.g. _Garry's parent's friend_), and tested on phrases with 3 common nouns (e.g. _Garry's parent's friend's enemy_). A number of examples is set aside as dev(elopment) data: once the model achieves good results on dev, the training is stopped.

As you see below, training a neural network proceeds in _epochs_. During each epoch, the model processes the training data and updates its weights to better match the model's prediction with the correct label. The degree of mismatch, based on the likelihood the model assigns to correct answers, is measured as _loss_; smaller loss values are better.

In [ ]:
!python3 LSTM_composition/recurrent_NN_sentence_classifier.py --num_rels 4 --runs 1 -b l --detail True --early True -c 4 --top_complexity_in_train 0.0 --arch GRU -df left_branching.txt

 The trained model's outputs are stored in the file whose name appears after the ```-df``` flag above. We can check that the model's interpretations are indeed correct:

In [ ]:
!cat left_branching.txt

We can also try training the same model with different data. What if we use a right branching construction instead of left branching (```-b r``` flag in the command instead of ```-b l```)?

In [ ]:
!python3 LSTM_composition/recurrent_NN_sentence_classifier.py --num_rels 4 --runs 1 -b r --detail True --early True -c 4 --top_complexity_in_train 0.0 --arch GRU -df right_branching.txt

As it turns out, right branching is difficult for a neural network that processes its inputs in a strict left-to-right fashion. The resulting accuracy is not 1 anymore but closer to chance level (which would be 0.25: the chance of guessing the correct referent out of 4). You can check the model's predictions if you like.

In [ ]:
!cat right_branching.txt

Now on from small toy examples to a large language model. We can use the freely available GPT2 model:

In [ ]:
!git clone https://github.com/nshepperd/gpt-2.git

In [ ]:
cd gpt-2/

Install the required packages:

In [ ]:
!pip3 install -r requirements.txt

Now we can download a pretrained GPT2 model with a relatively small number of weight parameters (117 million).

In [ ]:
!python3 download_model.py 117M

To put GPT-2 to a test, we can create a list of examples as text snippets that GPT-2 is then asked to complete. We use two simple prompts here. Feel free to add more examples if you are curious.

In [ ]:
prompts=[
    "Bill's mother is Ann. Ann's enemy is George. Bill's mother's enemy is",
    "The mother of Bill is Ann. The enemy of Ann is George. The enemy of the mother of Bill is"]

Now we can run define function ```run_on_prompts``` adapting code from a GPT-2 script ```interactive_conditional_samples.py```.

In [ ]:
cd src/

In [ ]:
!wget https://raw.githubusercontent.com/kovvalsky/SemDL/main/gpt2generation.py SemDL/

In [ ]:
from gpt2generation import run_on_prompts

The function will give GPT-2 three tries to generate the next word following the prompt. How often is model's guess correct?

In [ ]:
run_on_prompts('117M',prompts)

The example above was done on the smallest GPT-2 model. Normally, bigger neural models can yield better results. So we can also download the largest GPT-2 model:

In [ ]:
!python3 download_model.py 1558M

In [ ]:
run_on_prompts('1558M',prompts)